In [1]:
#This is for map user hover india state wise data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/map/user/hover/country/india/state"
Map_user_state_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'State':[], 'Year':[],'Quater':[],'District_name':[], 'Registered_Users_Count':[], 'Apps_Open':[]}

for i in Map_user_state_list:
    p_i=path+"/"+i
    Agg_yr=os.listdir(p_i)
    #delhi
    for j in Agg_yr:
        p_j=p_i+"/"+j
        #2018
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+"/"+k 
            #1.json
            Data = open(p_k,'r')
            D=json.load(Data)
            for district, values in D['data']['hoverData'].items():
                Name = district
                count = values['registeredUsers']
                open_app = values['appOpens']
                clm['District_name'].append(Name)
                clm['Registered_Users_Count'].append(count)
                clm['Apps_Open'].append(open_app)
                clm['State'].append(i)
                clm['Year'].append(j)
                clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Map_user_state=pd.DataFrame(clm)
print(Map_user_state)

                           State  Year  Quater  \
0      andaman-&-nicobar-islands  2018       1   
1      andaman-&-nicobar-islands  2018       1   
2      andaman-&-nicobar-islands  2018       1   
3      andaman-&-nicobar-islands  2018       2   
4      andaman-&-nicobar-islands  2018       2   
...                          ...   ...     ...   
20603                west-bengal  2024       4   
20604                west-bengal  2024       4   
20605                west-bengal  2024       4   
20606                west-bengal  2024       4   
20607                west-bengal  2024       4   

                           District_name  Registered_Users_Count  Apps_Open  
0      north and middle andaman district                     632          0  
1                 south andaman district                    5846          0  
2                      nicobars district                     262          0  
3      north and middle andaman district                     911          0  
4        

In [2]:
#You need to map your State names to the names used in the GeoJSON file (properties.ST_NM). Otherwise you will get blank figure.
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
Map_user_state['State'] = Map_user_state['State'].str.strip().str.lower()
Map_user_state['State'] = Map_user_state['State'].replace(state_name_map)

In [ ]:
#creating table to store the above data
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="xxx", #give your user name. Mostly its root
    password="xxx", #give your password
    database="xxx" #give any database name you want
)
cursor=connection.cursor()
query1 = """
        create table Map_User_State(
        State varchar(100),
        Year int,
        Quater int,
        District_name varchar(100),
        Registered_Users_Count float,
        Apps_Open float)
        """
cursor.execute(query1)

In [4]:
#storing the above in mysql database
#all data must be tuple and must be a list of tuples
data=[]
for index in Map_user_state.index:
    row = Map_user_state.loc[index]
    row = row.values
    row = row[0],float(row[1]),int(row[2]),row[3],int(row[4]),float(row[5])
    data.append(row)
query2 = "insert into Map_User_State values (%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [3]:
#doing data cleaning for Map_user_state
from sklearn.preprocessing import LabelEncoder
#first step check and remove null values
Map_user_state.isna().sum()
Map_user_state.info()
#Second do encoding and dummies to make all values int
enc_col = ['District_name']
dummy_col = ['Year','Quater','State']
encoder = LabelEncoder()
Map_user_state['encoded_district_name'] = encoder.fit_transform(Map_user_state['District_name'])
Map_user_state = pd.get_dummies(Map_user_state,columns=dummy_col,dtype=int)
#third removing outliners
Map_user_state['registered_users_count_std']=(Map_user_state['Registered_Users_Count']-Map_user_state['Registered_Users_Count'].mean())/Map_user_state['Registered_Users_Count'].std()
Map_user_state['apps_open_count_std']=(Map_user_state['Apps_Open']-Map_user_state['Apps_Open'].mean())/Map_user_state['Apps_Open'].std()
condition1 = Map_user_state['registered_users_count_std']<=3
condition2 = Map_user_state['registered_users_count_std']>=-3
condition3 = Map_user_state['apps_open_count_std']<=3
condition4 = Map_user_state['apps_open_count_std']>=-3
Map_user_state = Map_user_state[(condition1) & (condition2) & (condition3) & (condition4)]
print(Map_user_state)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20608 entries, 0 to 20607
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   State                   20608 non-null  object
 1   Year                    20608 non-null  object
 2   Quater                  20608 non-null  int64 
 3   District_name           20608 non-null  object
 4   Registered_Users_Count  20608 non-null  int64 
 5   Apps_Open               20608 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 966.1+ KB
                           District_name  Registered_Users_Count  Apps_Open  \
0      north and middle andaman district                     632          0   
1                 south andaman district                    5846          0   
2                      nicobars district                     262          0   
3      north and middle andaman district                     911          0   
4                 south andam

In [6]:
#plotting aggregated_transaction_insurance in the form of india map using plotly
import plotly.express as px

fig = px.choropleth(
    Map_user_state,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='Registered_Users_Count',
    color_continuous_scale='Reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [4]:
#This is for map user hover india state 2018
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/map/user/hover/country/india/2018"
Map_user_2018_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'Quater':[],'State_name':[], 'Registered_Users_Count':[], 'Apps_Open':[]}

for i in Map_user_2018_list:
    p_i=path+"/"+i
    Data = open(p_k,'r')
    D=json.load(Data)
    for district, values in D['data']['hoverData'].items():
        Name = district
        count = values['registeredUsers']
        open_app = values['appOpens']
        clm['State_name'].append(Name)
        clm['Registered_Users_Count'].append(count)
        clm['Apps_Open'].append(open_app)
        clm['Quater'].append(int(i.strip('.json')))
#Succesfully created a dataframe
Map_user_2018=pd.DataFrame(clm)
print(Map_user_2018)

    Quater                           State_name  Registered_Users_Count  \
0        1  south twenty four parganas district                 2525913   
1        1             purba bardhaman district                 1435548   
2        1              uttar dinajpur district                  922938   
3        1                   kalimpong district                   78277   
4        1                 murshidabad district                 2463675   
..     ...                                  ...                     ...   
87       4                  alipurduar district                  475688   
88       4           paschim bardhaman district                 1468252   
89       4                       nadia district                 1861738   
90       4                     birbhum district                 1114220   
91       4             purba medinipur district                 1717074   

    Apps_Open  
0   148514597  
1    84180749  
2    66212953  
3     4244012  
4   165497019  
.. 